In [112]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date
import re

In [134]:
result_list = []
for num in range(70, 600):
    url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL'
    params = {
        'page' : num
    }
    resp = requests.get(url, params = params)
    soup = BeautifulSoup(resp.content)
    tr_list = soup.find_all('tr')[1:]
    for tr in tr_list:
        time = tr.find_all('td')[0].text.strip()
        a = time.split('.')
        time2 = date(int(a[0]), int(a[1]), int(a[2]))
        price = tr.find_all('td')[1].text
        updown = tr.find_all('td')[3].text.rstrip("%").strip()
        
        new_dict = {
            'date' : time2,
            'price' : price,
            'upanddown' : updown
        }
        result_list.append(new_dict)

In [140]:
result_list

[{'date': datetime.date(2018, 8, 7), 'price': '1.44', 'upanddown': '-3.36'},
 {'date': datetime.date(2018, 8, 6), 'price': '1.49', 'upanddown': '0.00'},
 {'date': datetime.date(2018, 8, 3), 'price': '1.49', 'upanddown': '0.00'},
 {'date': datetime.date(2018, 8, 2), 'price': '1.49', 'upanddown': '-0.67'},
 {'date': datetime.date(2018, 8, 1), 'price': '1.50', 'upanddown': '-1.32'},
 {'date': datetime.date(2018, 7, 31), 'price': '1.52', 'upanddown': '+ 2.01'},
 {'date': datetime.date(2018, 7, 30), 'price': '1.49', 'upanddown': '0.00'},
 {'date': datetime.date(2018, 7, 27), 'price': '1.49', 'upanddown': '-0.67'},
 {'date': datetime.date(2018, 7, 26), 'price': '1.50', 'upanddown': '0.00'},
 {'date': datetime.date(2018, 7, 25), 'price': '1.50', 'upanddown': '0.00'},
 {'date': datetime.date(2018, 7, 24), 'price': '1.50', 'upanddown': '+ 1.35'},
 {'date': datetime.date(2018, 7, 23), 'price': '1.48', 'upanddown': '0.00'},
 {'date': datetime.date(2018, 7, 20), 'price': '1.48', 'upanddown': '0.00

In [141]:
result_df = pd.DataFrame(result_list)
result_df = result_df.set_index('date') # 이 과정이 없으면 merge가 안된다.
result_df.head()

,price,upanddown
date,,
2018-08-07,1.44,-3.36
2018-08-06,1.49,0.00
2018-08-03,1.49,0.00
2018-08-02,1.49,-0.67
2018-08-01,1.50,-1.32


In [90]:
result_df


,price,upanddown
date,,
2018-08-07,1.44,-3.36%
2018-08-06,1.49,0.00%
2018-08-03,1.49,0.00%
2018-08-02,1.49,-0.67%
2018-08-01,1.50,-1.32%
...,...,...
2003-09-05,3.67,-0.54%
2003-09-04,3.69,+ 1.10%
2003-09-03,3.65,-0.27%


In [91]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3710 entries, 2018-08-07 to 2003-09-01
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   price      3710 non-null   object
 1   upanddown  3710 non-null   object
dtypes: object(2)
memory usage: 87.0+ KB


In [43]:
result_df.to_csv("callrate.csv", mode='w')

In [142]:
fulldate = pd.DataFrame(columns = ['date'])
start = date(2004, 8, 20)
end = date(2019, 7, 19)
for n in range(int((end - start).days) + 1):
    fulldate.loc[n, 'date'] = start + timedelta(n)
fulldate

,date
0,2004-08-20
1,2004-08-21
2,2004-08-22
3,2004-08-23
4,2004-08-24
...,...
5442,2019-07-15
5443,2019-07-16
5444,2019-07-17
5445,2019-07-18


In [143]:
rate = fulldate.merge(result_df, on = 'date', how = 'outer')
rate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5687 entries, 0 to 5686
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       5687 non-null   object
 1   price      3710 non-null   object
 2   upanddown  3710 non-null   object
dtypes: object(3)
memory usage: 177.7+ KB


In [160]:
rate['price']=rate['price'].fillna(method='pad')
rate['upanddown']=rate['upanddown'].fillna(method='pad')
rate.to_csv("callrate_new.csv", mode='w')